# Voca Quiz Mid and High prompt - [  ] 작업

In [44]:
import json
from openai import OpenAI
from pydantic import BaseModel
import jinja2
import pandas as pd
from getpass import getpass
from openai.lib._parsing import type_to_response_format_param

api_key = getpass("OEPNAI_API_KEY")
client = OpenAI(api_key=api_key)

In [83]:
from jinja2 import Template

prompt_template = Template("""
[PERSONA]
You are an AI that processes and modifies string data.

[PURPOSE]
```
Identify the "WordMeaning" value inside the "korean_translation" string.
Wrap the "WordMeaning" value in double square brackets [[ ]].
```

[Example]
```
Input: 
    {
    "WordText": "accent",
    "WordMeaning": "강세",
    "PoSText": "명사",
    "fill_in_blank_quiz" : "She speaks with a unique ___.",
    "korean_translation" : "그녀는 독특한 강세로 말해요."
  }

Output:
    {
    "korean_translation_blank": "그녀는 독특한 [[강세]]로 말해요."
    }
```
                           
[Example]
```
Input: 
    {
    "WordText": "work out",
    "WordMeaning": "(일이) 잘 풀리다",
    "PoSText": "숙어/관용어",
    "fill_in_blank_quiz" : "I believe things will ___ soon.",
    "korean_translation" : "나는 곧 모든 일이 잘 풀릴 거라고 믿어요."
  }

Output:
    {
    "korean_translation_blank": "나는 곧 모든 일이 [[잘 풀릴 거라고]] 믿어요."
    }
```

[Input]
```
WordText : {{WordText}}
WordMeaning : {{WordMeaning}}
PoSText : {{PoSText}}
FILLINTHEBLANK : {{fill_in_blank_quiz}}
KOREANTRANSLATION : {{korean_translation}}
```
"""
)

In [100]:
prompt = prompt_template.render(
    WordText = "warm up",
    WordMeaning = "준비 운동을 하다",
    PoSText = "숙어/관용어",
    fill_in_blank_quiz = "I always ___ before jogging.",
    korean_translation = "나는 조깅하기 전에 준비 운동을 해요.",
)

In [101]:
print(prompt)


[PERSONA]
You are an AI that processes and modifies string data.

[PURPOSE]
```
Identify the "WordMeaning" value inside the "korean_translation" string.
Wrap the "WordMeaning" value in double square brackets [[ ]].
```

[Example]
```
Input: 
    {
    "WordText": "accent",
    "WordMeaning": "강세",
    "PoSText": "명사",
    "fill_in_blank_quiz" : "She speaks with a unique ___.",
    "korean_translation" : "그녀는 독특한 강세로 말해요."
  }

Output:
    {
    "korean_translation_blank": "그녀는 독특한 [[강세]]로 말해요."
    }
```
                           
[Example]
```
Input: 
    {
    "WordText": "work out",
    "WordMeaning": "(일이) 잘 풀리다",
    "PoSText": "숙어/관용어",
    "fill_in_blank_quiz" : "I believe things will ___ soon.",
    "korean_translation" : "나는 곧 모든 일이 잘 풀릴 거라고 믿어요."
  }

Output:
    {
    "korean_translation_blank": "나는 곧 모든 일이 [[잘 풀릴 거라고]] 믿어요."
    }
```

[Input]
```
WordText : warm up
WordMeaning : 준비 운동을 하다
PoSText : 숙어/관용어
FILLINTHEBLANK : I always ___ before jogging.
KOREANTRANSLATION : 

In [76]:
class QuizResponse(BaseModel):
    korean_translation_blank : str

In [70]:
response_format = type_to_response_format_param(QuizResponse)
response_format

{'type': 'json_schema',
 'json_schema': {'schema': {'properties': {'korean_translation_blank': {'title': 'Korean Translation Blank',
     'type': 'string'}},
   'required': ['korean_translation_blank'],
   'title': 'QuizResponse',
   'type': 'object',
   'additionalProperties': False},
  'name': 'QuizResponse',
  'strict': True}}

In [87]:
def completion(prompt : str) -> str:
    response = client.beta.chat.completions.parse(
        model = 'o3-mini',
        reasoning_effort='low',
        messages = [
            {"role" : "system", "content" : "You need to create natural parentheses in Korean sentences by matching the meaning of a word to its definition."},
            {"role" : "user", "content" : prompt}
        ],
        response_format = QuizResponse,
    )
    return response.choices[0].message.parsed

In [107]:
response = completion(prompt)

In [108]:
response_output = json.dumps(response.dict(), ensure_ascii=False, indent=2)
print(response_output)

{
  "korean_translation_blank": "나는 조깅하기 전에 [[준비 운동을 해요]]."
}
